# Entendimiento de los datos. Miguel Camargo

## Configuración e importe de paquetes
Se utilizará el paquete de pandas profiling para apoyar el análisis estadístico, y se importan los paquetes de python
necesarios

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType, StringType, IntegerType, DateType
from pyspark.sql.functions import udf, col, length, isnan, when, count
import pyspark.sql.functions as f
import os 
from datetime import datetime
from pyspark.sql import types as t
from pandas_profiling import ProfileReport
#import matplotlib.pyplot as plt
import numpy as np

C:\Users\estudiante\anaconda3\envs\Tutoriales\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configuración del controlador e inicio de sesion Spark

In [2]:
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

In [10]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)

spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

C:\Users\estudiante\anaconda3\envs\Tutoriales\lib\site-packages\pyspark\sql\context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [11]:
# Si quiere practicar la conexion con el servidor de base de datos:
db_connection_string = 'jdbc:mysql://157.253.236.116:8080/WWImportersTransactional'
# El usuario es su estudiante _i asignado y su contraseña la encontrará en el archivo excel de Coursera 
db_user = 'Estudiante_9_202214'
db_psswd = 'DEP1WHSE9G'

PATH='./'

### Conexión a fuente de datos y acceso a los datos

In [12]:
def obtener_dataframe_de_bd(db_connection_string, sql, db_user, db_psswd):
    df_bd = spark.read.format('jdbc')\
        .option('url', db_connection_string) \
        .option('dbtable', sql) \
        .option('user', db_user) \
        .option('password', db_psswd) \
        .option('driver', 'com.mysql.cj.jdbc.Driver') \
        .load()
    return df_bd

In [13]:
sql_movements = 'WWImportersTransactional.movimientosCopia'
movements = obtener_dataframe_de_bd(db_connection_string, sql_movements, db_user, db_psswd)

# Completitud del perfilamiento de datos

In [14]:
movements.show(10)
print("Registros Totales: "+str(movements.count()))
movements.printSchema()
movements.summary().show()

+---------------------+----------+-----------------+---------+---------+-----------+---------------+----------------+--------+
|TransaccionProductoID|ProductoID|TipoTransaccionID|ClienteID|InvoiceID|ProveedorID|OrdenDeCompraID|FechaTransaccion|Cantidad|
+---------------------+----------+-----------------+---------+---------+-----------+---------------+----------------+--------+
|               118903|       217|               10|    476.0|  24904.0|           |               |     Apr 25,2014|   -40.0|
|               286890|       135|               10|     33.0|  60117.0|           |               |     Dec 10,2015|    -7.0|
|               285233|       111|               10|    180.0|  59768.0|           |               |     Dec 04,2015|    -2.0|
|               290145|       213|               10|     33.0|  60795.0|           |               |     Dec 23,2015|    -3.0|
|               247492|        90|               10|     55.0|  51851.0|           |               |     Jul 27

## Cantidad de registros y cantidad de columnas por tipo de dato

El total de los registros es de 204292, existen 9 colomnas, todas son nulleables, 3 de tipo integer, 3 de tipo double, y 3 de tipo string.

## Significado de una fila

Una fila representa una transaccion que se ha realizado a un producto en especifico, se relaciona tanto la factura, el tipo de transaccion y el cliente de cada transaccion
Adicionalmente la cantidad que se uso en la transaccion y la fecha en la que se realizo la transaccion.

## Estadisticas descriptivas por columna

En las estadisticas se puede observar varios temas, uno de ellos es la por ejemplo la cantidad minima en las transacciones la cual es -360 y la maxima 67368.0, que el total de transacciones es de 719.4997650421946

# Completitud del análisis de la calidad de datos

In [25]:
def contar_vacios(df):
    resultados = []
    for c in df.columns:
        vacios = df.filter((df[c].isNull()) | (df[c]=="")).count()
        if vacios!=0:
            print('registros vacíos para la columna => '+c+': '+str( vacios ))
            resultados.append(vacios)
    return resultados

def cardinalidad(df):
    resultado = {}
    for c in df.columns:
        cardinalidad = df.select(col(c)).distinct().count()
        if cardinalidad>=df.count()*0.5:
            resultado[c] = cardinalidad
    return resultado

## Completitud

In [26]:
contar_vacios(movements)

registros vacíos para la columna => ProveedorID: 197182
registros vacíos para la columna => OrdenDeCompraID: 197182


[197182, 197182]

Podemos ver que en la tabla de movimientosCopia hay 197182 registros sin ProviderID y sin OrdenDeCompraID, esto genera inconsistencia en los datos, ademas de que se podria pensar que hay una falla en el proceso de almacenamiento que genera esta inconsistencia, como que en los posible formularios estan llegando vacios estos campos, ademas de que la tabla no deberia permitir nulos, lamentablemente permito nullos en todas las columnas

In [27]:
columnas_alta_cardinalidad_movemen = cardinalidad(movements)
columnas_alta_cardinalidad_movemen

{'TransaccionProductoID': 173659}

Respecto a cardinalidad, ninguna de las columna tienen una cardinalidad superior al 50%.

## Unicidad

In [28]:
unique = movements.select(col('TransaccionProductoID')).distinct().count()
total = movements.count()
print('Registros unicos por ID: '+str(unique))
print('Registros totales: '+str(total))
print('Diferencia: '+str(total - unique))

Registros unicos por ID: 173659
Registros totales: 204292
Diferencia: 30633


Con el siguiente resultado podemos llegar a la conclusion que la llave primaria no es el ID de la transaccion, esto genera bastantes problemas, tales como que no podemos realziar consultas rapidas a estas tabla, o no podemos hacer unions por un id unico.

## Consistencia y Validez

In [30]:
regex = "[0-2]\d{3}-(0[1-9]|1[0-2])-(0[1-9]|[1-2][0-9]|3[0-1])"
formatCheck = movements.filter(movements["FechaTransaccion"].rlike(regex))
print("Total de movimientos: "+str(total)+", cantidad de movimientos con formato valido: "+str(formatCheck.count()))
print('Movimientos con formato invalido:: '+str(total - formatCheck.count()))
movements.select(col("FechaTransaccion")).show()

Total de movimientos: 204292, cantidad de movimientos con formato valido: 140038
Movimientos con formato invalido:: 64254
+----------------+
|FechaTransaccion|
+----------------+
|     Apr 25,2014|
|     Dec 10,2015|
|     Dec 04,2015|
|     Dec 23,2015|
|     Jul 27,2015|
|     Sep 15,2014|
|     Aug 04,2015|
|     Feb 23,2015|
|     May 01,2015|
|     Jan 08,2016|
|     Mar 26,2014|
|     Jul 31,2015|
|     Sep 02,2014|
|     Mar 15,2016|
|     May 28,2016|
|     Sep 09,2015|
|     May 23,2014|
|     Aug 20,2014|
|     Jan 21,2015|
|     Dec 29,2015|
+----------------+
only showing top 20 rows



Las especificaciones tecnicas de negocio afirman que el formato de las fechas debe ser YYYY-MM-DD HH:MM:SS, aplicando una regex se peude observar que hay 64254 que no cumplen con esta regla, lo cual no esta correcto, y puede generar problemas para hacer filtrados por la columna de fechas e incongruencias en las consultas.

# Conclusión del entendimiento de datos

## Justificación de si es posible resolver los análisis basados en tableros de control propuestos.

Luego de los diferentes analisis que se llevaron a cabo se puede concluir que esta tabla no posee la caracteristicas minimas necesiaras para ser plasmada en un tablero de control, no esta normalizada, tiene formatos invalidos, tiene muchos valoes nulos de campos importante, esto generaria bastantes incongruencias en el tablero

## Incluir la síntesis de lo que representa la fila promedio de los datos analizados.

En sintesis una fila representa una transaccion que se ha realizado a un producto en especifico, se relaciona tanto la factura, el tipo de transaccion y el cliente de cada transaccion
Adicionalmente la cantidad que se uso en la transaccion y la fecha en la que se realizo la transaccion.

# Lista de preguntas o comentarios realizados a la organización

1. Preguntaria si la tabla al ser una copia, peude ser un backup de una base de datos vieja
2. Preguntaria si existe una tabla relacionando los tipo de movimientos.
3. Preguntaria si existe una tabla con los diferente proveedores ya que solo se usan 3
4. Preguntaria si una transaccion es unica, para poder determianr la llave primaria.
5. Preguntaria si en alguno momento el proceso de guardado de informacion presento problemas, eso justificaria los valores nulos
6. Preguntaria las fuentes de origenes de los datos, si es un formulario, como se le esta dando el formato a la fecha.